# ChatBot usando Self-Organizing Maps

### Equipe:
* Julio Sales
* Luiz Fernando Calabria
* Macio Matheus
* Tacio Nery
* Victor Outtes

In [8]:
import numpy as np
from sklearn import datasets
from time import sleep, time
import matplotlib.pyplot as plt
import skfuzzy as fuzz
import random, csv
from sklearn.feature_extraction.text import HashingVectorizer
%matplotlib inline

Classe com os metodos de tratamento de texto. Gera a base de treinamento convertida para array numerico. O metodo "transform_input" transforma um texto de entrada em array numerico.

In [1]:
class CHATBOT:
    vectorizer = None
    base_treinamento = None
    target_treinamento = None
    
    def __init__(self):
        # utilizando 10 caracteristicas
        self.vectorizer = HashingVectorizer(n_features=10)
        self.train()
        
    def load_data(self):
        # base de entrada
        return ["Oi", "Tudo bem?", "Qual a sua idade?", "Como encerro este chat?", "Adeus"]

    def train(self):
        data = self.load_data()
        # transforma os dados de entrada em arrays de numeros
        v = self.vectorizer.fit_transform(data)
        self.base_treinamento = np.array(v.toarray())
        # target
        self.target_treinamento = ["Seja bem vindo", "Tudo", "Minha idade e indefinida", "Digite adeus", "Ate logo"]

    def transform_input(self,text):
        return self.vectorizer.transform([text]).toarray()

Classe que define a rede SOM. Possui dois metodos de treinamento: o original e um que utiliza Fuzzy para definicao do ajuste de pesos da vizinhanca.

In [10]:
class SOM:
    wNodes = None # Peso dos Nos

    alpha0 = None  # Taxa de Aprendizado
    sigma0 = None  # Raio
    dataIn = None  # Dados
    dataOut = None # respostas do chatbot
    grid = None

    def __init__(self, dataIn, dataOut, grid=[10, 10], alpha=0.1, sigma=None):
        dim = dataIn.shape[1]
        self.wNodes = np.random.uniform(-1, 1, [grid[0], grid[1], dim])
        self.alpha0 = alpha
        if (sigma is None):
            self.sigma0 = max(grid) / 2.0
        else:
            self.sigma0 = sigma

        self.dataIn = np.asarray(dataIn)
        self.grid = grid
        self.dataOut = dataOut

    def train(self, maxIt=100, verbose=True, analysis=False, timeSleep=0.5):
        nSamples = self.dataIn.shape[0]
        m = self.wNodes.shape[0] #linha
        n = self.wNodes.shape[1] #coluna

        # Processamento (TEMPO)
        timeCte = (maxIt / np.log(self.sigma0))
        if analysis:
            print('timeCte = ', timeCte)

        timeInit = 0
        timeEnd = 0
        
        for epc in range(maxIt):
            alpha = self.alpha0 * np.exp(-epc / timeCte)
            sigma = self.sigma0 * np.exp(-epc / timeCte)

            if verbose:
                print('Epoca: ', epc, ' - Tempo de Processamento: ', (timeEnd - timeInit) * (maxIt - epc), ' seg')

            timeInit = time()

            for k in range(nSamples):

                # Nó Vencedor
                matDist = self.distance(self.dataIn[k, :], self.wNodes)

                posWin = self.getWinNodePos(matDist)

                deltaW = 0
                h = 0

                for i in range(m):
                    for j in range(n):
                        # Distância Entre Dois Nos
                        dNode = self.getDistanceNodes([i, j], posWin)

                        # Região de Vizinhança
                        h = np.exp((-dNode ** 2) / (2 * sigma ** 2))

                        # Atualizando os Pesos
                        deltaW = (alpha * h * (self.dataIn[k, :] - self.wNodes[i, j, :]))
                        self.wNodes[i, j, :] += deltaW

                        if analysis:
                            print('Epoca = ', epc)
                            print('Amostra = ', k)
                            print('-------------------------------')
                            print('alpha = ', alpha)
                            print('sigma = ', sigma)
                            print('h = ', h)
                            print('-------------------------------')
                            print('No vencedor = [', posWin[0], ', ', posWin[1], ']')
                            print('No atual = [', i, ', ', j, ']')
                            print('Distancia entre Nos = ', dNode)
                            print('deltaW = ', deltaW)
                            print('wNode antes = ', self.wNodes[i, j, :])
                            print('wNode depois = ', self.wNodes[i, j, :] + deltaW)
                            print('\n')
                            sleep(timeSleep)

            timeEnd = time()
            
    def trainFuzzy(self, maxIt=100, verbose=True, analysis=False, timeSleep=0.5):
        nSamples = self.dataIn.shape[0]
        m = self.wNodes.shape[0] #linha
        n = self.wNodes.shape[1] #coluna

        # Processamento (TEMPO)
        timeCte = (maxIt / np.log(self.sigma0))
        if analysis:
            print('timeCte = ', timeCte)

        timeInit = 0
        timeEnd = 0
        
        for epc in range(maxIt):
            alpha = self.alpha0 * np.exp(-epc / timeCte)
            sigma = self.sigma0 * np.exp(-epc / timeCte)

            if verbose:
                print('Epoca: ', epc, ' - Tempo de Processamento: ', (timeEnd - timeInit) * (maxIt - epc), ' seg')

            timeInit = time()

            for k in range(nSamples):

                # Nó Vencedor
                matDist = self.distance(self.dataIn[k, :], self.wNodes)

                posWin = self.getWinNodePos(matDist)

                deltaW = 0
                h = 0

                for i in range(m):
                    for j in range(n):
                        # Distância Entre Dois Nos
                        dNode = self.getDistanceNodes([i, j], posWin)

                        # Região de Vizinhança
                        pertinencia = self.pertinencia(dNode)
                        
                        h = pertinencia #np.exp((-dNode ** 2) / (2 * sigma ** 2))

                        # Atualizando os Pesos
                        deltaW = (alpha * h * (self.dataIn[k, :] - self.wNodes[i, j, :]))
                        self.wNodes[i, j, :] += deltaW

                        if analysis:
                            print('Epoca = ', epc)
                            print('Amostra = ', k)
                            print('-------------------------------')
                            print('alpha = ', alpha)
                            print('sigma = ', sigma)
                            print('h = ', h)
                            print('-------------------------------')
                            print('No vencedor = [', posWin[0], ', ', posWin[1], ']')
                            print('No atual = [', i, ', ', j, ']')
                            print('Distancia entre Nos = ', dNode)
                            print('deltaW = ', deltaW)
                            print('wNode antes = ', self.wNodes[i, j, :])
                            print('wNode depois = ', self.wNodes[i, j, :] + deltaW)
                            print('\n')
                            sleep(timeSleep)

            timeEnd = time()

    # Método para calcular a distância entre a entrada e seus pesos
    def distance(self, a, b):
        return np.sqrt(np.sum((a - b) ** 2, 2, keepdims=True))

    # Método que retorna a distância entre dois nós

    def getDistanceNodes(self, n1, n2):
        n1 = np.asarray(n1)
        n2 = np.asarray(n2)
        return np.sqrt(np.sum((n1 - n2) ** 2))

    # Método que retorna a posição do Nó vencedor
    def getWinNodePos(self, dists):
        arg = dists.argmin()
        m = dists.shape[0]
        return arg // m, arg % m

    # Método que retornar o centróide dos dados de entrada
    def getCentroid(self, data):
        data = np.asarray(data)
        N = data.shape[0]
        centroids = list()

        for k in range(N):
            matDist = self.distance(data[k, :], self.wNodes)
            centroids.append(self.getWinNodePos(matDist))

        return centroids

    # Método para salvar os Pesos
    def saveTrainedSOM(self, fileName='SOMTreinado.csv'):
        np.savetxt(fileName, self.wNodes)
    # Método para da um load nos Pesos já treinado - Utilizar para Teste
    def setTrainedSOM(self, fileName):
        self.wNodes = np.loadtxt(fileName)
    
    # Metodo para, dado um exemplo, informar qual neuronio responde
    def predict(self, sample):
        x,y = self.getWinNodePos(self.distance(sample, self.wNodes))
        return y,x
    # Metodo para verificar o grau de pertinencia de um neuronio a outro, dada a distancia entre eles. 
    # Distancia zero gera pertinencia 1.
    def pertinencia(self, distancia):
        universo = np.arange(0, 100, 1)
        return fuzz.interp_membership(universo, fuzz.gaussmf(universo, 0.0, 5), distancia)
    # Metodo para predizer a resposta do Bot dado uma entrada. Utiliza o "predict" com a entrada e calcula
    # a classe associada ao dado de treinamento mais parecido.
    def predictClass(self, sample):
        distancias = []
        for dado in self.dataIn:
            distancias.append(self.getDistanceNodes(self.predict(dado),self.predict(sample)))
        indiceMenor = distancias.index(min(distancias))
        return self.dataOut[indiceMenor]

        

### Execucao

Criando uma instancia do chatbot

In [13]:
chat = CHATBOT()

Instancia duas SOM (som e som2). Uma eh treinada com o algoritmo original e a outra com Fuzzy.

In [15]:
# definicao do grid da SOM
tamanho = [30, 30]
som = SOM(chat.base_treinamento, chat.target_treinamento, tamanho, alpha=0.3)
# treinamento
som.train(maxIt=30,verbose=False)

som2 = SOM(chat.base_treinamento, chat.target_treinamento, tamanho, alpha=0.3)
# treinamento Fuzzy
som2.trainFuzzy(maxIt=30,verbose=False)

### ChatBot sendo executado

Deixa o sistema escutando o teclado ate a tecla ENTER ser pressionada. Transforma o texto do input em um array numerico e chama o predictClass para dar a resposta do bot.

In [21]:
while True:
    pergunta = input('PERGUNTA: ')
    resposta = som.predictClass(chat.transform_input(pergunta))
    print('BOT: ', resposta)
    if resposta == 'Ate logo':
        break

PERGUNTA: OI
BOT:  Seja bem vindo
PERGUNTA: Tudo bem?
BOT:  Tudo
PERGUNTA: Qual sua idade?
BOT:  Minha idade e indefinida
PERGUNTA: Qual seu time?
BOT:  Digite adeus
PERGUNTA: Ola?
BOT:  Tudo
PERGUNTA: adeus
BOT:  Ate logo
